In [44]:
import requests
import pandas as pd
import os 
import json
import http.client
import time

In [46]:
# request league data

url = "https://v3.football.api-sports.io/leagues"

payload={}
headers = {
  'x-rapidapi-key': 'a9cc039ebd059e2c63f6a40d741e82a8',
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

response_leagues = requests.request("GET", url, headers=headers, data=payload)

json_leagues = response_leagues.json()
df = pd.DataFrame(json_leagues['response'])

df_leagues = pd.DataFrame(df.loc[:,'league'].values.tolist()).iloc[:,0:3]

df_country = pd.DataFrame(df.loc[:,'country'].values.tolist()).iloc[:,0:2]
df_country = df_country.rename(columns={'name':'country_name', 'code':'country_code'})

df_leagues = pd.concat([df_leagues, df_country], axis=1)
df_leagues

,id,name,type,country_name,country_code
0,4,Euro Championship,Cup,World,None
1,21,Confederations Cup,Cup,World,None
2,61,Ligue 1,League,France,FR
3,144,Jupiler Pro League,League,Belgium,BE
4,71,Serie A,League,Brazil,BR
...,...,...,...,...,...
920,906,Reserve League,League,Argentina,AR
921,907,Primera Division,League,Cuba,CU
922,908,AFF U23 Championship,Cup,World,None
923,909,MLS Next Pro,League,USA,US


In [48]:
# top five and next five countries according to UEFA country coefficients

top_five = ['England', 'Spain', 'Germany', 'Italy', 'France']
next_five = ['Portugal', 'Netherlands', 'Austria', 'Scotland', 'Russia']

for country in top_five:
    print(country, ': ', country in set(df_leagues['country_name'].values))
    
for country in next_five:
    print(country, ': ', country in set(df_leagues['country_name'].values))

England :  True
Spain :  True
Germany :  True
Italy :  True
France :  True
Portugal :  True
Netherlands :  True
Austria :  True
Scotland :  True
Russia :  True


In [49]:
leagues_temp = pd.DataFrame()

# adding two top-tier leagues from top five countries
for country in top_five:
    top_2_leagues = df_leagues[(df_leagues['country_name'] == country) & (df_leagues['type'] == 'League')].iloc[0:2]
    leagues_temp = leagues_temp.append(top_2_leagues)

# adding one top tier league from the rest of five countries
for country in next_five:
    top_league = df_leagues[(df_leagues['country_name'] == country) & (df_leagues['type'] == 'League')].iloc[0:1]
    leagues_temp = leagues_temp.append(top_league)

df_leagues = leagues_temp
df_leagues

,id,name,type,country_name,country_code
5,39,Premier League,League,England,GB
110,40,Championship,League,England,GB
10,140,La Liga,League,Spain,ES
253,141,Segunda Division,League,Spain,ES
6,78,Bundesliga 1,League,Germany,DE
60,79,Bundesliga 2,League,Germany,DE
7,135,Serie A,League,Italy,IT
254,136,Serie B,League,Italy,IT
2,61,Ligue 1,League,France,FR
16,62,Ligue 2,League,France,FR


In [12]:
df_leagues.to_csv('C:/coding/Python/practice/football_analysis/2nd analysis - underestimated players/leagues.csv', index=False)

In [50]:

fixtures_goal = pd.DataFrame()
league_list = df_leagues['id'].values.tolist()

for league in league_list:
    
    # 'URL' + '?' + 'query parameter'
    # use '&' to add multiple queries
    url_fixture = "https://v3.football.api-sports.io/fixtures?league={}&season=2021".format(league)

    payload={}
    headers = {
      'x-rapidapi-key': 'a9cc039ebd059e2c63f6a40d741e82a8',
      'x-rapidapi-host': 'v3.football.api-sports.io'
    }

    response_fixtures = requests.request("GET", url_fixture, headers=headers, data=payload)

    json_fixtures = response_fixtures.json()
    df_fixtures = pd.DataFrame(json_fixtures['response'])

    fixture_id = pd.DataFrame(df_fixtures.loc[:,'fixture'].values.tolist())
    fixture_id = fixture_id[['id']]
    fixture_id['league_id'] = league
    fixture_id = fixture_id.rename(columns={'id':'fixture_id'})

    fixture_teams = pd.DataFrame(df_fixtures.loc[:,'teams'].values.tolist())

    fixture_teams_home = pd.DataFrame(fixture_teams['home'].values.tolist())
    fixture_teams_home = fixture_teams_home[['id','name']]
    fixture_teams_home = fixture_teams_home.rename(columns={'id':'home_id','name':'home_name'})

    fixture_teams_away = pd.DataFrame(fixture_teams['away'].values.tolist())
    fixture_teams_away = fixture_teams_away[['id','name']]
    fixture_teams_away = fixture_teams_away.rename(columns={'id':'away_id','name':'away_name'})

    fixture_goals = pd.DataFrame(df_fixtures.loc[:,'goals'].values.tolist())
    fixture_goals = fixture_goals.rename(columns={'home':'home_goals','away':'away_goals'})

    df_fixtures = pd.concat([fixture_id, fixture_teams_home, fixture_teams_away, fixture_goals], axis=1)

    fixtures_goal = fixtures_goal.append(df_fixtures)
    
    
# use parameter encoding='utf-8-sig' to be able to display letters with accents etc with no error
fixtures_goal.to_csv('C:/coding/Python/practice/football_analysis/2nd analysis - underestimated players/fixtures_goal.csv', index=False, encoding='utf-8-sig')

In [51]:
len(fixtures_goal['fixture_id'].values.tolist())

5287

In [52]:
fixtures_goal

,fixture_id,league_id,home_id,home_name,away_id,away_name,home_goals,away_goals
0,710556,39,55,Brentford,42,Arsenal,2,0
1,710557,39,44,Burnley,51,Brighton,1,2
2,710558,39,49,Chelsea,52,Crystal Palace,3,0
3,710559,39,45,Everton,41,Southampton,3,1
4,710560,39,46,Leicester,39,Wolves,1,0
...,...,...,...,...,...,...,...,...
239,711535,235,1994,Khimki,558,Spartak Moscow,2,1
240,862967,235,1087,Ska-khabarovsk,1994,Khimki,1,0
241,862968,235,1994,Khimki,1087,Ska-khabarovsk,3,0
242,862969,235,1080,Orenburg,1078,FC UFA,2,2


In [56]:
# prediction data

fixture_id_list = fixtures_goal['fixture_id'].values.tolist()
# fixture_id_list = [719472, 719473]

pred_criteria = ['form', 'att', 'def', 'poisson_distribution', 'h2h', 'goals', 'total']
fixtures_prediction = pd.DataFrame()

for fixture in fixture_id_list:
    
    conn_pred = http.client.HTTPSConnection("v3.football.api-sports.io")

    headers_pred = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': "a9cc039ebd059e2c63f6a40d741e82a8"
        }

    conn_pred.request("GET", "/predictions?fixture={}".format(fixture), headers=headers_pred)

    res_pred = conn_pred.getresponse()
    data_pred = res_pred.read()

    json_pred = json.loads(data_pred.decode("utf-8"))
    df_pred = pd.DataFrame(json_pred['response'])
    
    # temporary dictionary to keep the data until the data gets appended to the final prediction dataframe
    pred_data = {'home': [], 'away': []}
    
    for team in ['home', 'away']:    
        
        team_id = df_pred['teams'][0][team]['id']
        team_name = df_pred['teams'][0][team]['name']
        
        for criteria in pred_criteria:
            
            # keep each data in the temporary dictionary above by home and away
            pred_data[team].append(df_pred['comparison'][0][criteria][team])
        
        # gather the team info and prediction data altogether and append them to the final prediction dataframe
        fixtures_prediction = fixtures_prediction.append(pd.concat(
            [
                pd.DataFrame(
                    [[fixture, team_id, team_name]],
                    columns=['fixture_id', 'team_id', 'team_name']),
                pd.DataFrame(
                    [pred_data[team]],
                    columns=pred_criteria)
            ], axis=1))

# use parameter encoding='utf-8-sig' to be able to display letters with accents etc with no error
fixtures_prediction.to_csv('C:/coding/Python/practice/football_analysis/2nd analysis - underestimated players/fixtures_prediction.csv', index=False, encoding='utf-8-sig')
fixtures_prediction

,fixture_id,team_id,team_name,form,att,def,poisson_distribution,h2h,goals,total
0,710556,55,Brentford,0%,0%,0%,0%,0%,25%,25.0%
0,710556,42,Arsenal,0%,0%,0%,0%,100%,75%,75.0%
0,710557,44,Burnley,0%,0%,0%,0%,50%,55%,52.5%
0,710557,51,Brighton,0%,0%,0%,0%,50%,45%,47.5%
0,710558,49,Chelsea,0%,0%,0%,0%,100%,82%,82.0%
...,...,...,...,...,...,...,...,...,...,...
0,862968,1087,Ska-khabarovsk,43%,20%,100%,0%,50%,50%,40.8%
0,862969,1080,Orenburg,0%,0%,100%,0%,71%,55%,63.0%
0,862969,1078,FC UFA,100%,100%,0%,0%,29%,45%,37.0%
0,862970,1078,FC UFA,90%,78%,29%,0%,38%,50%,57.0%


In [54]:
# get each player's stats for each fixtures

start_time = time.time()

fixture_id_list = fixtures_goal['fixture_id'].values.tolist()

player_stats_total = pd.DataFrame()

# fixtures that have no player stats data cause KeyError
key_error_fixtures = pd.DataFrame()

print('fixtures with no data (max. no printed: 10): ')

# select a fixture
for fixture in fixture_id_list:
        
    conn = http.client.HTTPSConnection("v3.football.api-sports.io")

    headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': "a9cc039ebd059e2c63f6a40d741e82a8"
        }

    conn.request("GET", "/fixtures/players?fixture={}".format(fixture), headers=headers)

    res = conn.getresponse()
    data = res.read()

    json_player_stats = json.loads(data.decode("utf-8"))
    df_player_stats = pd.DataFrame(json_player_stats['response'])
        
    # extract each player's data in various level and  them all into one table - player_stats_total

    #  0 for home team 1 for away team
    for i in [0,1]:
                
        try:
            
            # extract each player's data in various level
            for nth_player in range(len(df_player_stats.loc[i,'players'])):
                
                team_id = df_player_stats.loc[i,'team']['id']
                team_name = df_player_stats.loc[i,'team']['name']

                player_id = df_player_stats.loc[i,'players'][nth_player]['player']['id']
                player_name = df_player_stats.loc[i,'players'][nth_player]['player']['name']

                position = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['games']['position']
                mins_played = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['games']['minutes']
                rating = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['games']['rating']
                substitute = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['games']['substitute']

                offsides = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['offsides']

                shots_total = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['shots']['total']
                shots_on = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['shots']['on']
                try: 
                    shots_off = int(shots_total - shots_on)
                except TypeError:
                    shots_off = 0

                goals = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['goals']['total']
                assists = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['goals']['assists']
                saves = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['goals']['saves']

                passes_total = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['passes']['total']
                passes_key = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['passes']['key']
                passes_accuracy = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['passes']['accuracy']

                tackles = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['tackles']['total']
                blocks = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['tackles']['blocks']
                interceptions = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['tackles']['interceptions']

                duels_total= df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['duels']['total']
                duels_won = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['duels']['won']

                dribble_attempt = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['dribbles']['attempts']
                dribble_success = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['dribbles']['success']
                dribble_past = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['dribbles']['past']

                fouls_drawn = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['fouls']['drawn']
                fouls_committed = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['fouls']['committed']

                yellow_card = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['cards']['yellow']
                red_card = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['cards']['red']

                penalty_won = df_player_stats.loc[i,'players'][nth_player]['statistics'][0]['penalty']['won']


                # append extracted data into the dataframe

                player_stats_total = player_stats_total.append(pd.DataFrame
                ({
                'fixture_id':[fixture],
                'team_id':[team_id],
                'team_name':[team_name],

                'player_id':[player_id],
                'player_name':[player_name],

                'position':[position],
                'mins_played':[mins_played],
                'rating':[rating],
                'substitute':[substitute],

                'offsides':[offsides],

                'shots_total':[shots_total],
                'shots_on':[shots_on],
                'shots_off':[shots_off],

                'goals':[goals],
                'assists':[assists],
                'saves':[saves],

                'passes_total':[passes_total],
                'passes_key':[passes_key],
                'passes_accuracy':[passes_accuracy],

                'tackles':[tackles],
                'blocks':[blocks],
                'interceptions':[interceptions],

                'duels_total':[duels_total],
                'duels_won':[duels_won],

                'dribble_attempt':[dribble_attempt],
                'dribble_success':[dribble_success],
                'dribble_past':[dribble_past],

                'fouls_drawn':[fouls_drawn],
                'fouls_committed':[fouls_committed],

                'yellow_card':[yellow_card],
                'red_card':[red_card],

                'penalty_won':[penalty_won]
                }),ignore_index=True)

        except KeyError:
            if len(key_error_fixtures) < 10:
                print(fixture)
                key_error_fixtures = key_error_fixtures.append(pd.DataFrame({'fixture id with no data': [fixture]}),ignore_index=True)
                break
            else:
                key_error_fixtures = key_error_fixtures.append(pd.DataFrame({'fixture id with no data': [fixture]}),ignore_index=True)
                break
            # if there is a fixture that causes a KeyError, print the id - max 10 id's - and add it to a separate dataframe
            # a fixture that causes a KeyError doesn't contain player stats data even in API

# change data type
# can't convert None to integer using astype

player_stats_total[['fixture_id','team_id', 'player_id']] = player_stats_total[['fixture_id','team_id', 'player_id']].astype(int)
player_stats_total['substitute'] = player_stats_total['substitute'].astype(bool)

numeric_columns = player_stats_total.loc[:,'offsides':'penalty_won'].columns.tolist()
player_stats_total[numeric_columns] = player_stats_total[numeric_columns].fillna(0)

# bring some more info from other dataframes on the fixtures with no data 

key_error_fixtures = pd.merge(left=key_error_fixtures, right=fixtures_goal, how='left', \
                              left_on='fixture id with no data', right_on='fixture_id')
key_error_fixtures = pd.merge(left=key_error_fixtures, right=df_leagues, how='left', \
                              left_on='league_id', right_on='id')
key_error_fixtures = key_error_fixtures[['fixture id with no data', 'league_id', 'name', 'country_name']]
key_error_fixtures = key_error_fixtures.rename(columns={'name': 'league_name'})

print("'player_stats_total' and 'key_error_fixtures' complete")
print("number of fixtures with no data: ", key_error_fixtures.shape[0])
print("--- %s seconds ---" % (time.time() - start_time))

# use parameter encoding='utf-8-sig' to be able to display letters with accents etc with no error
# csv must be converted into unicode text file again and then imported into sql (on truncation - ignore) in order to preserve players' name
player_stats_total.to_csv('C:/coding/Python/practice/football_analysis/2nd analysis - underestimated players/player_stats_total.csv', index=False, encoding='utf-8-sig')
key_error_fixtures.to_csv('C:/coding/Python/practice/football_analysis/2nd analysis - underestimated players/fixtures_no_data.csv', index=False, encoding='utf-8-sig')

fixtures with no data (max. no printed: 10): 
862066
862298
717925
717926
717927
717928
717929
717930
717932
717934
'player_stats_total' and 'key_error_fixtures' complete
number of fixtures with no data:  218
--- 13600.980523347855 seconds ---


In [55]:
player_stats_total

,fixture_id,team_id,team_name,player_id,player_name,position,mins_played,rating,substitute,offsides,...,duels_total,duels_won,dribble_attempt,dribble_success,dribble_past,fouls_drawn,fouls_committed,yellow_card,red_card,penalty_won
0,710556,55,Brentford,19465,David Raya,G,90,7.9,False,0,...,1,1,0,0,0,0,0,0,0,0
1,710556,55,Brentford,1119,Kristoffer Ajer,D,71,6.6,False,0,...,4,2,1,1,0,0,0,0,0,0
2,710556,55,Brentford,19124,Pontus Jansson,D,90,7,False,0,...,1,0,0,0,0,0,0,0,0,0
3,710556,55,Brentford,19789,Ethan Pinnock,D,90,7.3,False,0,...,10,5,1,1,2,1,0,0,0,0
4,710556,55,Brentford,19352,Sergi Canós,M,90,8.6,False,0,...,8,7,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206293,862970,1080,Orenburg,119207,Artem Shabolin,M,None,None,True,0,...,0,0,0,0,0,0,0,0,0,0
206294,862970,1080,Orenburg,43499,Nikolai Prudnikov,F,None,None,True,0,...,0,0,0,0,0,0,0,0,0,0
206295,862970,1080,Orenburg,131571,Nikolay Titkov,M,21,6.6,True,0,...,6,3,1,0,0,0,1,1,0,0
206296,862970,1080,Orenburg,43590,Arshak Koryan,F,16,6.7,True,0,...,1,1,0,0,0,0,0,0,0,0


In [56]:
key_error_fixtures

,fixture id with no data,league_id,league_name,country_name
0,862066,62,Ligue 2,France
1,862298,62,Ligue 2,France
2,717925,219,Erste Liga,Austria
3,717926,219,Erste Liga,Austria
4,717927,219,Erste Liga,Austria
...,...,...,...,...
213,860848,179,Premiership,Scotland
214,861284,179,Premiership,Scotland
215,861285,179,Premiership,Scotland
216,861998,179,Premiership,Scotland
